
## ARMA Forecasting

**Functions**

arma_forecaster, cov_nw, ols

### Exercise 60
Produce 1-step forecasts from your preferred model in the previous exercise, as well as a random-walk model. 

1. Are the forecasts objectively accurate?
2. Compare these forecasts to the random walk models using MSE and MAE.


### Exercise 61

Produce 3-step forecasts from the models selected in the previous exercises as well as a random walk model.

1. Compare these forecasts to the random walk models using MSE and MAE.

Note: Use 50% of the sample to estimate the model and 50% to evaluate it.
